# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Цель нашего исследования - построить модель для задачи классификации, которая выберет подходящий тариф, с максимально большим значением *accuracy* (минимум 0.75). Предобработка данных не понадобится — она уже была проведена ранее.

Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Приблизительный план исследования:
- изучить представленные данные
- разбить данные на три выборки: обучающую, валидационную и тестовую
- исследовать три модели: решающее дерево, случайный лес и логистическую регрессию - и выбрать модель с самым высоким значением accuracy
- проверить наиболее удачную модель на тестовой выборке и выяснить значение accuracy на тестовой выборке

## Откроем и изучим файл

In [ ]:
#импортируем необходимые для работы библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv') #прочитаем файл и сохраним его в переменной df

In [ ]:
df.head(10) #выведем первые 10 строчек датафрейма

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
df.info() #выведем основную информацию о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:** мы прочитали файл, посмотрели на данные, содержащиеся в нём, а также на основную информацию о нём. Предобработка данных не требуется, так как она уже была проведена ранее.

## Разобьём данные на выборки

В данном исследовании спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Разобьём исходные данные в соотношении 3:1:1.

In [ ]:
features = df.drop(['is_ultra'], axis=1) #в качестве параметров передадим все столбцы, кроме 'is_ultra'
target = df['is_ultra'] #в качестве целевого значения возьмём столбец 'is_ultra'

Сначала разобьём данные на обучающую и тестовую выборку. features_train - обучающие параметры, features_test - тестовые параметры, target_train - обучающее целевое значение, target_test - тестовое целевое значение. Так как мы разбиваем данные в соотношении 60%-20%-20%, и сейчас мы отделяем тестовую выборку, в которую входит и валидационная, в параметр test_size мы передадим 0.2+0.2=0.4. Также зададим параметр псевдослучайных чисел random_state.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.4,
                                                                            random_state=12345
                                                                           )

Посмотрим на размер обучающей выборки:

In [ ]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


Теперь разобьём тестовую часть выборки на тестовую и валидационную: в параметр test_size мы передадим 0.5, поскольку сейчас мы работаем с 40% исходных данных (features_test и target_test), и нам нужно разделить их пополам и получить изначальное сооношение 3:1:1.

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test,
                                                                            target_test,
                                                                            test_size=0.5,
                                                                            random_state=12345
                                                                           )

Посмотрим на размеры валидационной выборки:

In [ ]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


Посмотрим на размеры тестовой выборки:

In [ ]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


Проверим, что сумма размеров получившихся выборок соответсвует размеру исходного датафрейма:

In [ ]:
1928 + 643 * 2

3214

**Вывод:** мы разбили исходные данные на три выборки: обучающую, валидационную и тестовую - в соотношении 3:1:1. Мы разделили данные таким образом, потому что в данном исследовании спрятанной тестовой выборки нет.

## Исследуем модели

Исследуем три модели с различными гиперпараметрами, чтобы увидеть, где точность будет выше всего, и выберем наиболее удачную модель.

### Решающее дерево

In [ ]:
best_model_tree = None
best_depth_tree = 0
best_result_tree = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_tree.fit(features_train, target_train)
    predictions_valid_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(predictions_valid_tree, target_valid)
    if best_result_tree < result_tree:
        best_model_tree = model_tree
        best_depth_tree = depth
        best_result_tree = result_tree

print('Лучшее значение max_depth:', best_depth_tree)
print('Лучшее значение accuracy:', best_result_tree)

Лучшее значение max_depth: 3
Лучшее значение accuracy: 0.7853810264385692


### Случайный лес

In [ ]:
best_model_forest = None
best_est_forest = 0
best_depth_forest = 0
best_result_forest = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model_forest.fit(features_train, target_train)
        predictions_valid_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(predictions_valid_forest, target_valid)
        if best_result_forest < result_forest:
            best_model_forest = model_forest
            best_est_forest = est
            best_depth_forest = depth
            best_result_forest = result_forest

print('Лучшее значение n_estimators:', best_est_forest)
print('Лучшее значение max_depth:', best_depth_forest)
print('Лучшее значение accuracy:', best_result_forest)

Лучшее значение n_estimators: 8
Лучшее значение max_depth: 8
Лучшее значение accuracy: 0.80248833592535


### Логистическая регрессия

In [ ]:
model_regr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regr.fit(features_train, target_train)
predictions_valid_regr = model_regr.predict(features_valid)
accuracy_regr = accuracy_score(predictions_valid_regr, target_valid)

print('Значение accuracy:', accuracy_regr)

Значение accuracy: 0.7107309486780715


**Вывод:** мы проверили три модели - решающее дерево, случайный лес и логистическую регрессию - и выяснили, что самое высокое значение accuracy (0.80248833592535) даёт **случайный лес** с гиперпараметрами n_estimators (количеством деревьев) и max_depth (глубиной дерева), равными 8. На втором месте значение accuracy (0.7853810264385692) у решающего дерева с гиперпараметром max_depth равным 3. На третьем месте значение accuracy (0.7107309486780715) у логистической регрессии. Следует выбрать модель случайного леса для проверки на тестовой выборке.

## Проверим модель на тестовой выборке

Проверим наиболее удачную модель случайного леса на тестовой выборке:

In [ ]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators = 8, max_depth = 8)
model_forest.fit(features_train, target_train)
predictions_test_forest = model_forest.predict(features_test)
result_test_forest = accuracy_score(predictions_test_forest, target_test)
print('Значение accuracy для тестовой модели:', result_test_forest)

Значение accuracy для тестовой модели: 0.7962674961119751


**Вывод:** мы проверили модель случайного леса на тестовой выборке и достигли значения accuracy 0.7962674961119751. Это значение выше порогового 0.75, поэтому можно сказать, что модель обучена успешно.

## Вывод

Мы изучили данные о поведении клиентов оператора мобильной связи «Мегалайн»:

1) Изучили представленные данные. Предобработка для данных не требовалась, так как она уже была произведена заранее.

2) Разбили данные на три выборки: обучающую, валидационную и тестовую - в соотношении 3:1:1. Мы разделили данные таким образом, потому что в данном исследовании спрятанной тестовой выборки нет.

3) Проверили три модели - решающее дерево, случайный лес и логистическую регрессию - и выяснили, что самое высокое значение accuracy (0.80248833592535) даёт случайный лес с гиперпараметрами n_estimators (количеством деревьев) и max_depth (глубиной дерева), равными 8. На втором месте значение accuracy (0.7853810264385692) у решающего дерева с гиперпараметром max_depth равным 3. На третьем месте значение accuracy (0.7107309486780715) у логистической регрессии. На основании этого мы выбрали модель случайного леса для проверки на тестовой выборке.

4) Проверили модель случайного леса на тестовой выборке и достигли значения accuracy 0.7962674961119751. Это значение выше порогового 0.75, поэтому можно сказать, что модель обучена успешно.